In [ ]:
# !pip install emoji
!pip install demoji
!pip install spacy
!pip install razdel
!pip install natasha
!python -m spacy download ru_core_news_lg

In [ ]:
import spacy
import pandas as pd
import demoji

In [ ]:
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from natasha import Segmenter
from razdel import sentenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df = pd.read_excel('/content/Датасет.xlsx', sheet_name='Лист1')

In [ ]:
df1 = df.copy()

In [ ]:
# df.iloc[:, [3,4,5]]
df1 = df1[['id', 'name(название)', 'responsibilities(Должностные обязанности)', 'requirements(Требования к соискателю)', 'terms(Условия)']]
df1.head(3)


,id,name(название),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия)
0,9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,NaN,NaN
1,9495850,Арматурщик,Гнутье арматурной стали на механических станка...,NaN,NaN
2,9495851,Арматурщик,Вязка арматуры.,NaN,NaN


In [ ]:
# Заполнение пропусков
df1 = df1.fillna('***')

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 5 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   id                                         999 non-null    int64 
 1   name(название)                             999 non-null    object
 2   responsibilities(Должностные обязанности)  999 non-null    object
 3   requirements(Требования к соискателю)      999 non-null    object
 4   terms(Условия)                             999 non-null    object
dtypes: int64(1), object(4)
memory usage: 39.1+ KB


In [ ]:
# Преобразование к строковому типу и удаление лишних пробелов в начале и конце строки
for col in df1.columns:
  df1[col] = df1[col].astype(str)
  df1[col] = df1[col].apply(lambda x: x.strip())
  df1[col] = df1[col].apply(lambda x: ' '.join(x.split()))
  df1[col] = df1[col].str.replace('- ', '')

In [ ]:
# Заменяем эмоджи на точки
def remove_emoji(text):
    return demoji.replace(text, '.')

In [ ]:
for col in df1.columns:
  df1[col] = df1[col].apply(remove_emoji)
  # убираем точки в начале строки
  df1[col] = df1[col].apply(lambda x: x if x[0] !='.' else x[1:])

In [ ]:
df1.head()

,id,name(название),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия)
0,9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,***,***
1,9495850,Арматурщик,Гнутье арматурной стали на механических станка...,***,***
2,9495851,Арматурщик,Вязка арматуры.,***,***
3,2,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,***
4,9388857,Арматурщик,Вахта в город Москва. Обязанности: армирование...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...


In [ ]:
# Удаление информации из колонки, которая есть в других колонках
def remove_text(source, target1, target2):
  text = source.replace(target1, '').replace(target2, '')
  text =  '***' if text == '' else text
  text = text if text[0] !='.' else text[1:]
  return text

In [ ]:
df1['responsibilities'] = df1.apply(lambda x: remove_text(x['responsibilities(Должностные обязанности)'], x['requirements(Требования к соискателю)'], x['terms(Условия)']), axis = 1).astype(str)

In [ ]:
df1.head()

,id,name(название),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),responsibilities
0,9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,***,***,Работа на строительных площадках. Очистных соо...
1,9495850,Арматурщик,Гнутье арматурной стали на механических станка...,***,***,Гнутье арматурной стали на механических станка...
2,9495851,Арматурщик,Вязка арматуры.,***,***,Вязка арматуры.
3,2,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,***,Выполнение работ по сборке и вязке арматурных...
4,9388857,Арматурщик,Вахта в город Москва. Обязанности: армирование...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,Вахта в город Москва. Обязанности: армирование...


In [ ]:
# Ставим точки перед словами с большой буквы
def dotted_text(text):
  pattern1 = r'(?<=\s)([А-ЯЁ][а-яё]*)'   # re.findall('[А-Я][а-я]+', s)
  pattern2 = r'\. \.'
  replacement1 = r'.\1'
  replacement2 = r'. '
  text1 = re.sub(pattern1, replacement1, text)
  text2 = re.sub(pattern2, replacement2, text1)
  return text2

In [ ]:
for col in df1.columns:
  df1[col] = df1[col].apply(lambda x: dotted_text(x))
  df1[col] = df1[col].apply(lambda text: '***' if text == '' else text)
  df1[col] = df1[col].apply(lambda text: text if text[0] !='.' else text[1:])
  # df1[col].replace(': \.| \.', '. ', regex=True, inplace=True)
  # df1[col].replace('•', '', inplace=True)

In [ ]:
df1.head()

,id,name(название),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),responsibilities
0,9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,***,***,Работа на строительных площадках. Очистных соо...
1,9495850,Арматурщик,Гнутье арматурной стали на механических станка...,***,***,Гнутье арматурной стали на механических станка...
2,9495851,Арматурщик,Вязка арматуры.,***,***,Вязка арматуры.
3,2,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,***,. Выполнение работ по сборке и вязке арматурны...
4,9388857,Арматурщик,Вахта в город. Москва. Обязанности: армировани...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,Вахта в город. Москва. Обязанности: армировани...


In [ ]:
df1.to_csv('df1.csv')